<a href="https://colab.research.google.com/github/NiloyPurkait/GSoC-2020/blob/master/RDF2Txt_GraphEncoder_RNNDecoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install sentencepiece

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals


import pickle
import math
import os
import re
import unicodedata
from functools import reduce
import numpy as np
from tqdm import tqdm
import time
import sentencepiece as spm

import tensorflow as tf
tf.compat.v1.enable_eager_execution()
#import tensorflow.contrib.slim as slim


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:

#tiny number
_NEG_INF = -1e9

## Preprocessing Helper functions 


In [0]:



def _tensorize(vocab, text):
    """
    Function to convert texts into number sequences first, and then
    add padding. Basically, tensorising them.
    :param vocab: The vocab which is used to lookup ids
    :type vocab: tf.tokenizer obj
    :param text: A list of sentences or a text file
    :type text: list
    :return: tensorised text data
    :rtype: tf.tensor
    """
    tensor = vocab.texts_to_sequences(text)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                           padding='post')

    return tensor


In [0]:
def max_length(tensor):
    return max(len(t) for t in tensor)

In [0]:

def padding(tensor, max_length):
    """
    Pads the given tensor to a maximum sequence length along
    axis 1.
    for ex -
    let the tensor be [1,2,3,4] if th given max_length is 5
    the tensor becomes [1,2,34,0]
    Mostly used to pad the target sentences of the multilingual
    model and the node_list of all models,

    :param tensor:A tf tensor
    :type tensor:tf.tensor
    :param max_length:Dimension along axis 1, of the new tensor
    :type max_length:int
    :return:The padded tensor
    :rtype:tf tensor.
    """

    padding = tf.constant([[0, 0], [0, max_length - tensor.shape[1]]])
    padded_tensor = tf.pad(tensor, padding, mode='CONSTANT')

    return padded_tensor

In [0]:
# from src.DataLoader imports

## Dataset Loading Functions

In [0]:
def LoadGatDataset(train_path, eval_path, test_path, srv_vocab,
                   tgt_vocab, opt, sentencepiece, lang, num_examples=None):
    train_ = {}
    eval_ = {}
    test_ = {}

    if opt == 'reif':
        # load the train and eval datasets
        with open(train_path, 'rb') as f:
            train_set = pickle.load(f)
        with open(eval_path, 'rb') as f:
            eval_set = pickle.load(f)
        with open(test_path, 'rb') as f:
            test_set = pickle.load(f)

        # load vocab
        if sentencepiece == 'True':
            sp = spm.SentencePieceProcessor()
            sp.load(tgt_vocab)
        with open(srv_vocab, 'rb') as f:
            src_vocab = pickle.load(f)

        train_input, train_tgt = zip(*train_set)
        eval_input, eval_tgt = zip(*eval_set)
        (train_nodes, train_labels, train_node1, train_node2) = zip(*train_input)
        (eval_nodes, eval_labels, eval_node1, eval_node2) = zip(*eval_input)
        (test_nodes, test_labels, test_node1, test_node2) = zip(*test_set)

        train_["train_node_tensor"] = _tensorize(src_vocab, train_nodes)
        train_["train_label_tensor"] = _tensorize(src_vocab, train_labels)
        train_["train_node1_tensor"] = _tensorize(src_vocab, train_node1)
        train_["train_node2_tensor"] = _tensorize(src_vocab, train_node2)

        eval_["eval_node_tensor"] = _tensorize(src_vocab, eval_nodes)
        eval_["eval_label_tensor"] = _tensorize(src_vocab, eval_labels)
        eval_["eval_node1_tensor"] = _tensorize(src_vocab, eval_node1)
        eval_["eval_node2_tensor"] = _tensorize(src_vocab, eval_node2)

        test_["test_node_tensor"] = _tensorize(src_vocab, test_nodes)
        test_["test_label_tensor"] = _tensorize(src_vocab, test_labels)
        test_["test_node1_tensor"] = _tensorize(src_vocab, test_node1)
        test_["test_node2_tensor"] = _tensorize(src_vocab, test_node1)

        #######exp######
        if sentencepiece == 'True':
            train_tgt_tensor = [sp.encode_as_ids(w) for w in train_tgt]
            train_["train_tgt_tensor"] = tf.keras.preprocessing.sequence.pad_sequences(train_tgt_tensor, padding='post')
            eval_tgt_tensor = [sp.encode_as_ids(w) for w in eval_tgt]
            eval_["eval_tgt_tensor"] = tf.keras.preprocessing.sequence.pad_sequences(eval_tgt_tensor, padding='post')
            target_vocab = sp
        else:
            train_tgt_tensor = src_vocab.texts_to_sequences(train_tgt)
            train_["train_tgt_tensor"] = tf.keras.preprocessing.sequence.pad_sequences(train_tgt_tensor, padding='post')
            eval_tgt_tensor = src_vocab.texts_to_sequences(eval_tgt)
            eval_["eval_tgt_tensor"] = tf.keras.preprocessing.sequence.pad_sequences(eval_tgt_tensor, padding='post')
            target_vocab = src_vocab

        return (train_, eval_, test_, src_vocab, target_vocab, max_length(train_tgt_tensor))

    else:
        # load the train and eval datasets
        with open(train_path, 'rb') as f:
            train_set = pickle.loads(f)
        with open(eval_path, 'rb') as f:
            eval_set = pickle.loads(f)
        # load vocab
        with open(srv_vocab, 'rb') as f:
            src_vocab = pickle.loads(f)

        train_input, train_tgt = zip(*train_set)
        eval_input, eval_tgt = zip(*eval_set)
        (train_adj, train_nodes, train_roles, train_edges) = zip(*train_input)
        (eval_adj, eval_nodes, eval_roles, eval_edges) = zip(*eval_input)

        train_node_tensor = _tensorize(src_vocab, train_nodes)
        train_role_tensor = _tensorize(src_vocab, train_roles)
        train_edges_tensor = _tensorize(src_vocab, train_edges)
        train_tgt_tensor = _tensorize(src_vocab, train_tgt)

        eval_node_tensor = _tensorize(src_vocab, eval_nodes)
        eval_role_tensor = _tensorize(src_vocab, eval_roles)
        eval_edges_tensor = _tensorize(src_vocab, eval_edges)

        return (train_adj, train_node_tensor, train_role_tensor, train_edges_tensor, train_tgt_tensor, eval_adj,
                eval_node_tensor, eval_role_tensor, eval_edges_tensor, src_vocab, max_length(train_tgt_tensor))




In [0]:
def GetGATDataset(train_path, eval_path,
                  test_path, src_vocab,
                  tgt_vocab, opt,
                  sentencepiece, lang,
                  set=None):
  
    if opt == 'reif':
        (train, eval, test, src_vocab, tgt_vocab, max_length_targ) = LoadGatDataset(train_path,
                                                                                    eval_path,
                                                                                    test_path, src_vocab,
                                                                                    tgt_vocab, opt,
                                                                                    sentencepiece, lang)

        node_tensor = padding(train["train_node_tensor"], 16)
        label_tensor = padding(train["train_label_tensor"], 16)
        node1_tensor = padding(train["train_node1_tensor"], 16)
        node2_tensor = padding(train["train_node2_tensor"], 16)

        eval_nodes = padding(eval["eval_node_tensor"], 16)
        eval_labels = padding(eval["eval_label_tensor"], 16)
        eval_node1 = padding(eval["eval_node1_tensor"], 16)
        eval_node2 = padding(eval["eval_node2_tensor"], 16)

        test_nodes = padding(test["test_node_tensor"], 16)
        test_labels = padding(test["test_label_tensor"], 16)
        test_node1 = padding(test["test_node1_tensor"], 16)
        test_node2 = padding(test["test_node2_tensor"], 16)

        print('\nTrain Tensor shapes (nodes, labels, node1, node2, target) : ')
        print(node_tensor.shape, label_tensor.shape, node1_tensor.shape, node2_tensor.shape, train["train_tgt_tensor"].shape)
        print('\nEval Tensor shapes (nodes, labes, node1, node2) : ')
        print(eval_nodes.shape, eval_labels.shape, eval_node1.shape, eval_node2.shape, eval["eval_tgt_tensor"].shape)
        print('\nTest Tensor shapes (nodes, labes, node1, node2) : ')
        print(test_nodes.shape, test_labels.shape, test_node1.shape, test_node2.shape)

        TRAIN_BUFFER_SIZE = len(train["train_tgt_tensor"])
        EVAL_BUFFER_SIZE = len(eval["eval_tgt_tensor"])
        BATCH_SIZE = batch_size
        steps_per_epoch = len(train["train_tgt_tensor"]) // BATCH_SIZE
        src_vocab_size = len(src_vocab.word_index) + 1
        if sentencepiece == 'True':
            tgt_vocab_size = tgt_vocab.get_piece_size()
        else:
            tgt_vocab_size = len(tgt_vocab.word_index) + 1

        dataset_size = train["train_tgt_tensor"].shape[0]

        dataset = tf.data.Dataset.from_tensor_slices((node_tensor, label_tensor,
                                                      node1_tensor, node2_tensor, train["train_tgt_tensor"])).shuffle(TRAIN_BUFFER_SIZE)
        dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

        eval_set = tf.data.Dataset.from_tensor_slices((eval_nodes, eval_labels,
                                                       eval_node1, eval_node2, eval["eval_tgt_tensor"])).shuffle(EVAL_BUFFER_SIZE)
        eval_set = eval_set.batch(BATCH_SIZE, drop_remainder=True)

        test_set = tf.data.Dataset.from_tensor_slices((test_nodes, test_labels,
                                                       test_node1, test_node2))
        test_set = test_set.batch(BATCH_SIZE, drop_remainder=True)

        if set == None:
            return (dataset, eval_set, test_set, TRAIN_BUFFER_SIZE, BATCH_SIZE, steps_per_epoch,
                    src_vocab_size, src_vocab, tgt_vocab_size, tgt_vocab,
                    max_length_targ, dataset_size)
        elif set == 'test':
            return (test_set, TRAIN_BUFFER_SIZE, BATCH_SIZE, steps_per_epoch,
                    src_vocab_size, src_vocab, tgt_vocab_size, tgt_vocab)

  



## Load processed Dataset

In [0]:

train_path = '/content/gdrive/My Drive/data/processed_graphs/eng/gat/reif_train'
eval_path = '/content/gdrive/My Drive/data/processed_graphs/eng/gat/reif_eval'
test_path = '/content/gdrive/My Drive/data/processed_graphs/eng/gat/reif_test'
src_vocab = '/content/gdrive/My Drive/data/processed_graphs/eng/gat/reif_src_vocab'
tgt_vocab =  '/content/gdrive/My Drive/data/processed_graphs/eng/gat/train_vocab.model'

opt = 'reif'
sentencepiece = True
lang = 'eng'
batch_size = 16

In [12]:
(dataset, eval_set, test_set, BUFFER_SIZE, BATCH_SIZE, steps_per_epoch,
     src_vocab_size, src_vocab, tgt_vocab_size, tgt_vocab, max_length_targ, dataset_size) = GetGATDataset(train_path, eval_path,
                                                                                                     test_path, src_vocab,
                                                                                                      tgt_vocab, opt,
                                                                                                      sentencepiece, lang)


Train Tensor shapes (nodes, labels, node1, node2, target) : 
(34352, 16) (34352, 16) (34352, 16) (34352, 16) (34352, 82)

Eval Tensor shapes (nodes, labes, node1, node2) : 
(34352, 16) (34352, 16) (34352, 16) (34352, 16) (34352, 82)

Test Tensor shapes (nodes, labes, node1, node2) : 
(1600, 16) (1600, 16) (1600, 16) (1600, 16)


## Probe the loaded dataset

In [0]:
print('Training batches have %d tensors'%len(next(iter(dataset))))

In [14]:
tgt_vocab_size

(10013, 10013)

In [15]:
 dataset_size, BUFFER_SIZE, BATCH_SIZE, steps_per_epoch

(34352, 34352, 16, 2147)

In [17]:
def least_frequent(word_count_dic, fq=200):
   return sorted([(i,word_count_dic[i]) for i in word_count_dic.keys() if word_count_dic[i] <= fq],
       key = lambda x : x[1],
       reverse=True)[:5]

def most_frequent(word_count_dic, fq=1000):
  return sorted([(i,word_count_dic[i]) for i in word_count_dic.keys() if word_count_dic[i] >= fq],
       key = lambda x : x[1],
       reverse=True)[:5]


[('3800.0', 200),
 ('raúl_fernando_sendic_rodríguez', 200),
 ('"100305.0"(minutes)', 200),
 ('david_scott', 200),
 ('rashid_rakhimov', 200)]

In [0]:

least_frequent(tgt_vocab.word_counts, fq=200)

In [18]:


most_frequent(tgt_vocab.word_counts, fq=1000)

[('a_zero', 203362),
 ('a_one', 203362),
 ('.', 125116),
 ('the', 102384),
 ('is', 71868)]

## Implement Encoder Module Layers:
To assemble the encoder modules, we must first define the following layers:
- Shared Embedding Layer
- Graph Attention Layer
- Feed Forward Layer


### Define Embedding Layer

In [0]:

class EmbeddingSharedWeights(tf.keras.layers.Layer):
    """Calculates input embeddings and pre-softmax linear with shared weights."""

    def __init__(self, vocab_size, hidden_size):
        """Specify characteristic parameters of embedding layer.

        Args:
          vocab_size: Number of tokens in the embedding. (Typically ~32,000)
          hidden_size: Dimensionality of the embedding. (Typically 512 or 1024)
        """
        super(EmbeddingSharedWeights, self).__init__()
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size

    def build(self, input_shape):
        """Build embedding layer."""
        with tf.name_scope("embedding_and_softmax"):
            # Create and initialize weights. The random normal initializer was chosen
            # arbitrarily, and works well.
            self.shared_weights = self.add_weight(
                "weights",
                shape=[self.vocab_size, self.hidden_size],
                dtype="float32",
                initializer=tf.random_normal_initializer(
                    mean=0., stddev=self.hidden_size ** -0.5))
        super(EmbeddingSharedWeights, self).build(input_shape)

    def get_config(self):
        return {
            "vocab_size": self.vocab_size,
            "hidden_size": self.hidden_size,
        }

    def call(self, inputs, mode="embedding"):
        """Get token embeddings of inputs.

        Args:
          inputs: An int64 tensor with shape [batch_size, length]
          mode: string, a valid value is one of "embedding" and "linear".
        Returns:
          outputs: (1) If mode == "embedding", output embedding tensor, float32 with
            shape [batch_size, length, embedding_size]; (2) mode == "linear", output
            linear tensor, float32 with shape [batch_size, length, vocab_size].
        Raises:
          ValueError: if mode is not valid.
        """
        if mode == "embedding":
            return self._embedding(inputs)
        elif mode == "linear":
            return self._linear(inputs)
        else:
            raise ValueError("mode {} is not valid.".format(mode))

    def _embedding(self, inputs):
        """Applies embedding based on inputs tensor."""
        with tf.name_scope("embedding"):
            # Create binary mask of size [batch_size, length]
            mask = tf.cast(tf.not_equal(inputs, 0), tf.float32)
            embeddings = tf.gather(self.shared_weights, inputs)
            embeddings *= tf.expand_dims(mask, -1)
            # Scale embedding by the sqrt of the hidden size
            embeddings *= self.hidden_size ** 0.5

            return embeddings

    def _linear(self, inputs):
        """Computes logits by running inputs through a linear layer.

        Args:
          inputs: A float32 tensor with shape [batch_size, length, hidden_size]
        Returns:
          float32 tensor with shape [batch_size, length, vocab_size].
        """
        with tf.name_scope("presoftmax_linear"):
            batch_size = tf.shape(inputs)[0]
            length = tf.shape(inputs)[1]

            x = tf.reshape(inputs, [-1, self.hidden_size])
            logits = tf.matmul(x, self.shared_weights, transpose_b=True)

            return tf.reshape(logits, [batch_size, length, self.vocab_size])



Define Graph Attention Layer


In [0]:
# Encoder layers = Embedding shared weights + GA Layer + FFN Layer


class GraphAttentionLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, dff, num_heads, reg_scale=0.001, rate=0.1):
        """
        Graph Attention Network Layer, takes input and returns embedded
        node features with self attention applied on the feature matrix
        """
        super(GraphAttentionLayer, self).__init__()
        self.in_dim = d_model
        self.out_dim = dff
        self.num_heads = num_heads
        self.dropout_rate = rate
        self.kernels = []
        self.biases = []
        self.attn_kernels = []

        self.lrelu = tf.keras.layers.LeakyReLU()
        self.dropout = tf.keras.layers.Dropout(rate)
        self.reg = tf.keras.regularizers.l2(l=reg_scale)

        for head in range(self.num_heads):
            kernel = self.add_weight(shape=(self.in_dim, self.out_dim),
                                     initializer='glorot_uniform',
                                     regularizer=self.reg,
                                     name='kernel_{}'.format(head))
            bias = self.add_weight(shape=(self.out_dim,),
                                   initializer='glorot_uniform',
                                   regularizer=self.reg,
                                   name='bias_{}'.format(head))
            self.kernels.append([kernel, bias])
            # Attention kernels
            attn_kernel_self = self.add_weight(shape=(self.out_dim, 1),
                                               initializer='glorot_uniform',
                                               regularizer=self.reg,
                                               name='attn_kernel_self_{}'.format(head))
            attn_kernel_neighs = self.add_weight(shape=(self.out_dim, 1),
                                                 initializer='glorot_uniform',
                                                 regularizer=self.reg,
                                                 name='attn_kernel_neigh_{}'.format(head))
            self.attn_kernels.append([attn_kernel_self, attn_kernel_neighs])

    def call(self, nodes):
        inputs = nodes

        outputs = []
        for head in range(self.num_heads):
            kernel = self.kernels[head]
            attention_kernel = self.attn_kernels[head]
            features = tf.keras.backend.dot(inputs, kernel[0])
            features = tf.add(features, kernel[1])
            attn_for_self = tf.keras.backend.dot(features, attention_kernel[0])
            attn_for_neighs = tf.keras.backend.dot(features, attention_kernel[1])
            # Attention head a(Wh_i, Wh_j) = a^T [[Wh_i], [Wh_j]]
            dense = tf.matmul(attn_for_self, attn_for_neighs, transpose_b=True)
            dense = self.lrelu(dense)

            # Mask values before activation (Vaswani et al., 2017)
            # mask_local = -10e9 * (1.0 - adj)
            # dense += mask_local

            # Apply softmax to get attention coefficients
            dense = tf.math.softmax(dense)  # (N x N)

            # Apply dropout to features and attention coefficients
            if self.trainable:
                dense = self.dropout(dense)  # (N x N)
                features = self.dropout(features)  # (N x F')

            # Linear combination with neighbors' features
            node_features = tf.matmul(dense, features)  # (N x F')
            outputs.append(node_features)

        output = tf.reduce_mean(tf.stack(outputs), axis=0)  # N x F')
        output = tf.nn.relu(output)

        return output



### Define Feed-Forward Layer

In [0]:
# Feed forward layer

class FeedForwardNetwork(tf.keras.layers.Layer):
    """Fully connected feedforward network."""

    def __init__(self, hidden_size, filter_size, relu_dropout):
        """Initialize FeedForwardNetwork.

        Args:
          hidden_size: int, output dim of hidden layer.
          filter_size: int, filter size for the inner (first) dense layer.
          relu_dropout: float, dropout rate for training.
        """
        super(FeedForwardNetwork, self).__init__()
        self.hidden_size = hidden_size
        self.filter_size = filter_size
        self.relu_dropout = relu_dropout

    def build(self, input_shape):
        self.filter_dense_layer = tf.keras.layers.Dense(
            self.filter_size,
            use_bias=True,
            activation=tf.nn.relu,
            name="filter_layer")
        self.output_dense_layer = tf.keras.layers.Dense(
            self.hidden_size, use_bias=True, name="output_layer")
        super(FeedForwardNetwork, self).build(input_shape)

    def get_config(self):
        return {
            "hidden_size": self.hidden_size,
            "filter_size": self.filter_size,
            "relu_dropout": self.relu_dropout,
        }

    def call(self, x, training):
        """Return outputs of the feedforward network.

        Args:
          x: tensor with shape [batch_size, length, hidden_size]
          training: boolean, whether in training mode or not.

        Returns:
          Output of the feedforward network.
          tensor with shape [batch_size, length, hidden_size]
        """
        # Retrieve dynamically known shapes
        batch_size = tf.shape(x)[0]
        length = tf.shape(x)[1]

        output = self.filter_dense_layer(x)
        if training:
            output = tf.nn.dropout(output, rate=self.relu_dropout)
        output = self.output_dense_layer(output)

        return output



## Assemble Graph Encoder module
We put together the encoder module using the defined layers above

In [0]:
# Graph Encoder Layer

class GraphEncoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, trainable, dff,
                 filter_size, reg_scale=0.001, rate=0.1):

        super(GraphEncoder, self).__init__()
        self.d_model = d_model
        self.num_layers = num_layers

        self.node_role_layer = tf.keras.layers.Dense(self.d_model, input_shape=(2 * d_model,))
        self.enc_layers = []
        for _ in range(num_layers):
            gat_layer = GraphAttentionLayer(d_model, dff, num_heads,
                                            reg_scale=reg_scale, rate=rate)
            ffn_layer = FeedForwardNetwork(dff, filter_size, rate)
            self.enc_layers.append([gat_layer, ffn_layer])

        self.dropout = tf.keras.layers.Dropout(rate)
        self.layernorm = tf.keras.layers.LayerNormalization()
        self.edge_layer = tf.keras.layers.Dense(self.d_model)
        self.trainable = trainable

    def call(self, node_tensor, label_tensor, node1_tensor, node2_tensor):
        # adding embedding and position encoding.

        edge_tensor = tf.concat([node1_tensor, node2_tensor], 2)
        edge_tensor = tf.cast(self.node_role_layer(edge_tensor), dtype=tf.float32)
        # node_tensor = tf.add(node_tensor, role_tensor)
        node_tensor *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        edge_tensor *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        # node_tensor += self.node_pos_enc[:, :node_seq_len, :]
        edges = self.edge_layer(tf.add(edge_tensor, label_tensor))


        for i, layer in enumerate(self.enc_layers):
            if i == 0:
                x = self.enc_layers[i][0](node_tensor)
                x = self.enc_layers[i][1](x, self.trainable)
                x += edges 
            else:
                shortcut = x
                x = self.enc_layers[i][0](x)
                x = self.enc_layers[i][1](x, self.trainable)
                x += edges
                x += shortcut

        return self.layernorm(x)

## Implement RNN Decoder Module Layers
To assemble the Decoder module, we will use the following layers:
- Embedding Layer
- Bidirectional GRU
- Bahanadu Attention Layer

In [0]:
# RNN Decoder =  embedding + birirect_GRU + BahanaduAttention

### Define Bahdanau Attention Layer

In [0]:
# Bahanadu Attention layer

class BahdanauAttention(tf.keras.Model):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, query, values):
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # we are doing this to perform addition to calculate the score
        hidden_with_time_axis = tf.expand_dims(query, 1)

        # score shape == (batch_size, max_length, hidden_size)
        score = self.V(tf.nn.tanh(
            self.W1(values) + self.W2(hidden_with_time_axis)))

        # attention_weights shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying score to self.V
        attention_weights = tf.nn.softmax(score, axis=1)

        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

### Assemble RNN decoder Module
We combine all the decoder layers under one class

In [0]:
# RNN Decoder
class RNNDecoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(RNNDecoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.forward_gru = tf.keras.layers.GRU(self.dec_units,
                                                    return_sequences=True,
                                                    return_state=True,
                                                    go_backwards=False,
                                                    recurrent_initializer='glorot_uniform')
        self.backward_gru = tf.keras.layers.GRU(self.dec_units,
                                                     return_sequences=True,
                                                     return_state=True,
                                                     go_backwards=True,
                                                     recurrent_initializer='glorot_uniform')
        self.gru = tf.keras.layers.Bidirectional(self.forward_gru, backward_layer=self.backward_gru,
                                                 merge_mode='ave')
        self.fc = tf.keras.layers.Dense(vocab_size)

        # used for attention
        self.attention = BahdanauAttention(self.dec_units)
        self.layernorm = tf.keras.layers.LayerNormalization()

    def call(self, x, hidden, enc_output):
        # enc_output shape == (batch_size, max_length, hidden_size)
        context_vector, attention_weights = self.attention(hidden, enc_output)

        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)

        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

        # passing the concatenated vector to the GRU
        output = self.gru(x)
        output, state = output[0], output[2]
        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))
        output = self.layernorm(output)

        # output shape == (batch_size, vocab)
        x = self.fc(output)

        return x, state, attention_weights



## Implement GAT Model
We put together the encoder and decoder modules from above to construct the Graph Attention model:

In [0]:
# GAT Model 

class GATModel(tf.keras.Model):
    """
    Model that uses Graph Attention encoder and RNN decoder (for now)
    """

    def __init__(self, 
                 enc_layers, enc_units,  emb_dim, num_heads,
                 hidden_size, filter_size, batch_size,  reg_scale,
                 dropout, src_vocab_size, tgt_vocab_size,
                 target_lang):
      
        super(GATModel, self).__init__()

        self.trainable = True
        self.emb_layer = EmbeddingSharedWeights(
            src_vocab_size, emb_dim)

        self.tgt_emb_layer = EmbeddingSharedWeights(
            tgt_vocab_size, emb_dim)

        self.encoder = GraphEncoder(enc_layers, emb_dim, num_heads, self.trainable,  hidden_size, # trainable=True
                                    filter_size, reg_scale=reg_scale, rate=dropout)
        self.decoder = RNNDecoder(tgt_vocab_size, emb_dim, enc_units, batch_size)
        self.vocab_tgt_size = tgt_vocab_size
        self.batch_size=batch_size
        self.num_heads = num_heads
        self.target_lang = target_lang
        self.loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
        self.hidden = tf.keras.layers.Dense(hidden_size)

    def __call__(self, nodes, labels, node1, node2, targ):
        """
        Puts the tensors through encoders and decoders
        :param adj: Adjacency matrices of input example
        :type adj: tf.tensor
        :param nodes: node features
        :type nodes: tf.tensor
        :param targ: target sequences
        :type targ: tf.tensor
        :return: output probability distribution
        :rtype: tf.tensor
        """
        node_tensor = tf.cast(self.emb_layer(nodes), dtype=tf.float32)
        label_tensor = tf.cast(self.emb_layer(labels), dtype=tf.float32)
        node1_tensor = tf.cast(self.emb_layer(node1), dtype=tf.float32)
        node2_tensor = tf.cast(self.emb_layer(node2), dtype=tf.float32)

        enc_output = self.encoder(node_tensor, label_tensor, node1_tensor, node2_tensor) # self.num_heads, self.encoder.trainable
        batch = enc_output.shape[0]
        self.enc_output_hidden = tf.reshape(enc_output, shape=[batch, -1])
        enc_hidden = self.hidden(self.enc_output_hidden)

        dec_input = tf.expand_dims([self.target_lang.word_index['<start>']] * self.batch_size, 1)
        loss = 0

        for t in range(1, targ.shape[1]):
            # pass encoder output to decoder
            predictions, dec_hidden, _ = self.decoder(dec_input, enc_hidden, enc_output)
            loss += loss_function(targ[:, t], predictions, self.loss_object)

            # using teacher forcing
            dec_input = tf.expand_dims(targ[:, t], 1)

        return predictions, dec_hidden, loss


## Define Loss function

In [0]:
def loss_function(real, pred, loss_object):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)


## Instantiate Model

In [0]:
'''
GATModel args: 

    enc_layers, enc_units,  emb_dim, num_heads,
    hidden_size, filter_size, batch_size,  reg_scale,
    dropout, src_vocab_size, tgt_vocab_size,
    target_lang

'''

model = GATModel(2, 64,  64, 2,
                 64, 16, 16,0.0,
                 0.2, src_vocab_size, tgt_vocab_size,
                 tgt_vocab)



## Test on sample output

In [32]:
#Example inputs and target batch for testing
nodes_, labels_, node1_, node2_, target_ = next(iter(dataset))

predictions, dec_hidden, loss = model(nodes_, labels_, node1_, node2_, target_)


for i in [predictions, dec_hidden]:
  print(i.shape) 

print(loss)


(16, 10013)
(16, 64)
tf.Tensor(225.53134, shape=(), dtype=float32)


## Define Train Step

In [0]:

def train_step(nodes, labels, node1, node2, targ):
    with tf.GradientTape() as tape:
        predictions, dec_hidden, loss = model(nodes, labels, node1, node2, targ)
        reg_loss = tf.reduce_sum(model.losses)
        loss += reg_loss
    batch_loss = (loss / int(targ.shape[1]))
    variables = model.trainable_variables
    gradients = tape.gradient(loss, variables)

    optimizer.apply_gradients(zip(gradients, variables))

    return batch_loss

## Define Eval step and helper functions for rogue_n score

In [0]:
def _get_ngrams(n, text):
    """Calculates n-grams.
    Args:
      n: which n-grams to calculate
      text: An array of tokens
    Returns:
      A set of n-grams
    """
    ngram_set = set()
    text_length = len(text)
    max_index_ngram_start = text_length - n
    for i in range(max_index_ngram_start + 1):
        ngram_set.add(tuple(text[i:i + n]))
    return ngram_set

In [0]:
def rouge_n(eval_sentences, ref_sentences, n=2):
    """Computes ROUGE-N f1 score of two text collections of sentences.
    Source: https://www.microsoft.com/en-us/research/publication/
    rouge-a-package-for-automatic-evaluation-of-summaries/
    Args:
      eval_sentences: The sentences that have been picked by the summarizer
      ref_sentences: The sentences from the reference set
      n: Size of ngram.  Defaults to 2.
    Returns:
      f1 score for ROUGE-N
    """

    f1_scores = []
    for eval_sentence, ref_sentence in zip(eval_sentences, ref_sentences):
        eval_ngrams = _get_ngrams(n, eval_sentence)
        ref_ngrams = _get_ngrams(n, ref_sentence)
        ref_count = len(ref_ngrams)
        eval_count = len(eval_ngrams)

        # Gets the overlapping ngrams between evaluated and reference
        overlapping_ngrams = eval_ngrams.intersection(ref_ngrams)
        overlapping_count = len(overlapping_ngrams)

        # Handle edge case. This isn't mathematically correct, but it's good enough
        if eval_count == 0:
            precision = 0.0
        else:
            precision = overlapping_count / eval_count

        if ref_count == 0:
            recall = 0.0
        else:
            recall = overlapping_count / ref_count

        f1_scores.append(2.0 * ((precision * recall) / (precision + recall + 1e-8)))

    # return overlapping_count / reference_count
    return np.mean(f1_scores, dtype=np.float32)

In [0]:
 def render(target):
    '''
    args:
      target_ : target tensor batch
      n = nth element in batch 

    usage:
      Takes in a tensor of (batch_size, vocab_size) and
      converts a batch instance into its string equivalent
    '''
    return [[tf.reduce_sum(i).numpy() for i in target_] for target_ in target]



In [0]:
def eval_step(EvalResultsFile, steps=5):

    reference = open(eval_path, 'r', encoding='ascii')
    model.trainable = False
    results = []
    ref_target = []
    eval_results = open(EvalResultsFile, 'w+')
    if steps is None:
        dev_set = eval_set
    else:
        dev_set = eval_set.take(steps)

    for (batch, (nodes, labels, node1, node2, targets)) in tqdm(enumerate(dev_set)):
        predictions = model(nodes, labels, node1,
                            node2, targets)
        pred = [(predictions[0].numpy().tolist())]


        for i in pred:
            sentences = tgt_vocab.sequences_to_texts(i)
            sentence = [j.partition("<start>")[2].partition("<end>")[0] for j in sentences]
            for w in sentence:
                eval_results.write((w + '\n'))
                ref_target.append(reference.readline())
                results.append(w)

    rogue = (rouge_n(results, ref_target))
    eval_results.close()
    model.trainable = True

    return rogue


## Set Model variables

In [0]:
epochs = 10
batch_size = 16

optimizer = tf.keras.optimizers.Adam()

loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

## Setup checkpoint manager

In [101]:

checkpoint_dir = './gdrive/My Drive/RDF_GAT_TF/training_checkpoints'

checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")

checkpoint = tf.train.Checkpoint(optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(checkpoint, checkpoint_dir, max_to_keep=5)


## Define Loss and accuracy objects

In [0]:
train_loss = tf.keras.metrics.Mean(name='train_loss')

train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

In [0]:
step=0
steps = epochs * steps_per_epoch
total_loss = 0
eval_file = './gdrive/My Drive/RDF_GAT_TF/eval_file.txt'

## Define training loop

In [0]:



for epoch in range(epochs):
    train_loss.reset_states()
    train_accuracy.reset_states()

    if ckpt_manager.latest_checkpoint:
        checkpoint.restore(ckpt_manager.latest_checkpoint)
        print('Latest checkpoint restored!!')

    # saving (checkpoint) the model every 2 epochs
    if (epoch + 1) % 2 == 0:
        print('- > Saving progress...')
        checkpoint.save(file_prefix = checkpoint_prefix)

    with tqdm(total=(38668 // batch_size)) as pbar:
        for (batch, (node_, label_, node1_, node2_, targ_)) in tqdm(enumerate(dataset)):
            start = time.time()
            step += 1

            batch_loss = train_step(node_, label_, node1_, node2_, targ_)
            print('Epoch {} Batch {} Batch Loss {:.4f} '.format(epoch, batch,
                                                                    batch_loss.numpy()))

            print('Time {} \n'.format(time.time() - start))
            pbar.update(1)

            #if batch % 5 ==0:
              #eval_step(eval_file)








  0%|          | 0/2416 [00:00<?, ?it/s]





0it [00:00, ?it/s]

Latest checkpoint restored!!







  0%|          | 1/2416 [00:02<2:00:09,  2.99s/it]





1it [00:02,  2.96s/it]

Epoch 0 Batch 0 Batch Loss 1.2630 
Time 2.89756441116333 








  0%|          | 2/2416 [00:05<1:54:37,  2.85s/it]





2it [00:05,  2.83s/it]

...

Epoch 0 Batch 1224 Batch Loss 1.4472 
Time 2.9708144664764404 








 51%|█████     | 1226/2416 [52:30<51:03,  2.57s/it]





1226it [52:30,  2.57s/it]

Epoch 0 Batch 1225 Batch Loss 1.2833 
Time 2.4480044841766357 








 51%|█████     | 1227/2416 [52:32<50:19,  2.54s/it]





1227it [52:32,  2.54s/it]

Epoch 0 Batch 1226 Batch Loss 1.1259 
Time 2.4462502002716064 

